# Techniques and metrics for evasion attack mitigation
    August 14 2025

In [24]:
import numpy as np

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import FastGradientMethod

# Step 1: Load dataset

In [25]:
data = load_wine()
X, y = data.data, data.target

# Standardize features for stability
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

print('Training Data:', X_train.shape, y_train.shape)
print('Training Data:', X_test.shape, y_test.shape)

Training Data: (142, 13) (142, 1)
Training Data: (36, 13) (36, 1)


In [26]:
enc = OneHotEncoder()
# Training
enc.fit(y_train)  
y_train_cat = enc.transform(y_train).toarray()

# Training
enc.fit(y_test)  
y_test_cat = enc.transform(y_test).toarray()

print(y_train[0:5])
print(y_train_cat[0:5])

[[2]
 [2]
 [1]
 [2]
 [0]]
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


# Step 2: Train a simple classifier

In [27]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model = SVC(C=1.0, kernel="rbf")

In [28]:
model.fit(X_train, y_train)

# Evaluate on clean test set
acc_clean = model.score(X_test, y_test)
print(f"Accuracy on clean test set: {acc_clean:.4f}")

Accuracy on clean test set: 1.0000


c:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Step 3: Wrap model for ART

In [29]:
classifier = SklearnClassifier(model=model)

classifier.fit(X_train, y_train_cat)

predictions = classifier.predict(X_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_cat, axis=1)) / len(y_test_cat)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 100.0%


# Step 4: Create adversarial attack (FGSM)

In [30]:
attack = FastGradientMethod(estimator=classifier, eps=0.1)

# Step 5: Generate adversarial examples

In [31]:
X_test_adv = attack.generate(x=X_test)

# Step 6: Evaluate on adversarial set

In [32]:
acc_adv = model.score(X_test_adv, y_test)
print(f"Accuracy on adversarial test set: {acc_adv:.4f}")

# Optional: Compare a few original vs adversarial samples
print("\nOriginal sample (first row):", X_test[0])
print("Adversarial sample (first row):", X_test_adv[0])

Accuracy on adversarial test set: 0.9722

Original sample (first row): [ 0.78980621  0.68550197  0.70724686 -1.28970717  1.14155847  0.64876424
  1.00469996 -1.54592178  0.12107098  0.01812906  0.01119018  1.05695159
  0.3124203 ]
Adversarial sample (first row): [ 0.6898062   0.78550196  0.80724686 -1.1897072   1.2415584   0.5487642
  0.9047     -1.6459218   0.02107098 -0.08187094 -0.08880982  0.9569516
  0.21242031]
